# Get Upcoming Event for Streamlit

In [1]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
from random import randint
import  random
import os
os.chdir('C:/Users/Travis/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')

from cmath import nan
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Getting NEXT Fights & Scraping URLS

In [2]:
def get_next_event_ufcstats():

    # scrape UFCStats website for next event

    url = 'http://www.ufcstats.com/statistics/events/upcoming'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('td', class_='b-statistics__table-col')
    event1_txt = soup.find('td', class_='b-statistics__table-col').text
    event_txt = event1_txt.replace('   ', '').replace('\n', '').strip()
    event_title = event_txt.split('  ')[0]
    event_date = event_txt.split('  ')[1]
    event1_url = event1.find('a')['href']
    data = pd.DataFrame({'event_title': [event_title], 'event_url': [event1_url], 'event_date': [event_date]})


    return data

In [3]:
next_event = get_next_event_ufcstats()
next_event

,event_title,event_url,event_date
0,UFC Fight Night: Lewis vs. Spivac,http://www.ufcstats.com/event-details/d26394fc...,"February 04, 2023"


#### Get all Data from Fighters in Next Event

In [4]:
def get_fighter_urls(event_details_url):

    # scrape UFCStats website for next event
    # Returns a dataframe with fighter names, fight urls, and fight number

    page = requests.get(event_details_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('tr', class_='b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click')
    n = 0
    next_event_data = pd.DataFrame()

    for event in events:
        fighters = events[n].find_all('p', class_='b-fight-details__table-text')
        fighter1 = fighters[0].text
        fighter1 = fighter1.replace('  ', '').replace('\n', '').strip()
        fighter2 = fighters[1].text
        fighter2 = fighter2.replace('  ', '').replace('\n', '').strip()
        fighter1_url = fighters[0].find('a')['href']
        fighter2_url = fighters[1].find('a')['href']
        next_event_data = next_event_data.append({'fighter1' :fighter1, 'fighter2:' : fighter2, 'fighter1_url': fighter1_url, 'fighter2_url':fighter2_url, 'fight#' : n+1}, ignore_index = True)
        n += 1

    return next_event_data

In [5]:
fighter_urls = get_fighter_urls(next_event['event_url'].values[0])
fighter_urls

,fighter1,fighter2:,fighter1_url,fighter2_url,fight#
0,Derrick Lewis,Serghei Spivac,http://www.ufcstats.com/fighter-details/d3df1a...,http://www.ufcstats.com/fighter-details/e2f6b2...,1
1,Da-Un Jung,Devin Clark,http://www.ufcstats.com/fighter-details/74f944...,http://www.ufcstats.com/fighter-details/cc7040...,2
2,Marcin Tybura,Blagoy Ivanov,http://www.ufcstats.com/fighter-details/c9cf75...,http://www.ufcstats.com/fighter-details/1788ee...,3
3,Dooho Choi,Kyle Nelson,http://www.ufcstats.com/fighter-details/e93b04...,http://www.ufcstats.com/fighter-details/31f808...,4
4,Yusaku Kinoshita,Adam Fugitt,http://www.ufcstats.com/fighter-details/21b5c8...,http://www.ufcstats.com/fighter-details/a01a62...,5
5,Jeka Saragih,Anshul Jubli,http://www.ufcstats.com/fighter-details/57186c...,http://www.ufcstats.com/fighter-details/a72a2a...,6
6,JeongYeong Lee,Yi Zha,http://www.ufcstats.com/fighter-details/d008d7...,http://www.ufcstats.com/fighter-details/3f11fd...,7
7,Toshiomi Kazama,Rinya Nakamura,http://www.ufcstats.com/fighter-details/148bb1...,http://www.ufcstats.com/fighter-details/d8c811...,8
8,SeungGuk Choi,HyunSung Park,http://www.ufcstats.com/fighter-details/adccbc...,http://www.ufcstats.com/fighter-details/b671bd...,9
9,Ji Yeon Kim,Mandy Bohm,http://www.ufcstats.com/fighter-details/a20f23...,http://www.ufcstats.com/fighter-details/297a2b...,10


In [6]:
def get_next_events_ufc_com(url):

    # make a looping version of get_next_events(url) 
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('div', class_='c-card-event--result__info')
    n = 0
    next_events_data = pd.DataFrame()

    for event in events:
        event_txt = events[n].text
        event_url = events[n].find('a')['href']
        event_url = 'https://www.ufc.com' + event_url
        event_title = event_txt.split('\n')[1]
        event_time = event_txt.split('/')[1]
        next_events_data = next_events_data.append({'event_title': event_title, 'event_url': event_url, 'event_date': event_time}, ignore_index=True)
        n += 1

    # only keep next event
    next_events_data = next_events_data.head(1)

    # add matchup
    main_fighter1 = next_events_data['event_title'].str.split('vs').str[0]
    main_fighter2 = next_events_data['event_title'].str.split('vs').str[1]

    next_events_data['main_fighter1'] = main_fighter1
    next_events_data['main_fighter2'] = main_fighter2
    return next_events_data

In [7]:
next= get_next_events_ufc_com('https://www.ufc.com/events')
next

,event_title,event_url,event_date,main_fighter1,main_fighter2
0,Lewis vs Spivac,https://www.ufc.com/event/ufc-fight-night-febr...,1:00 AM EST,Lewis,Spivac


#### Get Next Events from UFCStats

In [8]:
url = 'http://www.ufcstats.com/statistics/events/upcoming'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
    # get events
event = soup.findAll('i', class_ = 'b-statistics__table-content')
urls = soup.findAll('a', class_ = 'b-link b-link_style_black')
dates = soup.findAll('span', class_ = 'b-statistics__date')
n = 0
next_event_data = pd.DataFrame()

for i in event:
    # get event title, date, and url
    title = event[n].text
    title = title.replace('\n', '').strip()
    ti = title.split('  ')[0]
    date = dates[n].text.replace('\n', '').strip()
    #date = date.split('  ')[1]
    url = urls[n]['href']
    next_event_data = next_event_data.append({'event_title': ti, 'event_url': url, 'event_date': date}, ignore_index=True)
    n += 1


next_event_data['fighters'] = next_event_data['event_title'].str.split(': ').str[1]
next_event_data['main_fighter1'] = next_event_data['fighters'].str.split(' vs. ').str[0]
next_event_data['main_fighter2'] = next_event_data['fighters'].str.split(' vs. ').str[1]
next_event_data = next_event_data.head(1)
next_event_data


,event_title,event_url,event_date,fighters,main_fighter1,main_fighter2
0,UFC Fight Night: Lewis vs. Spivac,http://www.ufcstats.com/event-details/d26394fc...,"February 04, 2023",Lewis vs. Spivac,Lewis,Spivac


#### Get BestFightOdds Url

In [9]:
def my_search_bfo(fighter1, fighter2):

    # Get urls for bestfightodds for events from GOOGLE, as it is more consistant than bfo search

    try:
        time.sleep(5)
        url = 'https://www.google.com/search?query=site:bestfightodds.com ' + fighter1 + ' ' + fighter2
        header = {
                    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
                    "X-Requested-With": "XMLHttpRequest"
    }
        r = requests.get(url, headers = header)
        # get all links
        soup = BeautifulSoup(r.text, 'html.parser')
        links = soup.find_all('a')
        # keep only events
        goog_response = [n['href'] for n in links if 'events' in n['href']]
        # clean up the link so it is only the url
        goog_response = [n.replace('/url?q=', '') for n in goog_response]
        # cut off the begining up until https
        goog_response = [n[n.find('https'):] for n in goog_response]
        # cut off the end after the &
        goog_response = [n[:n.find('&')] for n in goog_response]
        # keep only the first one
        goog_response = goog_response[0]
        return goog_response
    except:
        return np.nan


In [10]:
# add the urls to the dataframe
next['bfo_url'] = next.apply(lambda x: my_search_bfo(x['main_fighter1'], x['main_fighter2']), axis=1)

In [11]:
next

,event_title,event_url,event_date,main_fighter1,main_fighter2,bfo_url
0,Lewis vs Spivac,https://www.ufc.com/event/ufc-fight-night-febr...,1:00 AM EST,Lewis,Spivac,https://www.bestfightodds.com/events/ufc-fight...


#### Add ufc_stats url

In [12]:
next['ufcstats_url'] = next_event_data['event_url']
next['ufc_stats_event_title'] = next_event_data['event_title']
next['ufc_stats_date'] = next_event_data['event_date']

In [13]:
next

,event_title,event_url,event_date,main_fighter1,main_fighter2,bfo_url,ufcstats_url,ufc_stats_event_title,ufc_stats_date
0,Lewis vs Spivac,https://www.ufc.com/event/ufc-fight-night-febr...,1:00 AM EST,Lewis,Spivac,https://www.bestfightodds.com/events/ufc-fight...,http://www.ufcstats.com/event-details/d26394fc...,UFC Fight Night: Lewis vs. Spivac,"February 04, 2023"


In [14]:
next.to_csv('data/final/events/upcoming_events_urls/' + str(next['event_title'].values[0]) + '.csv', index=False)

## Streamlit Functionality

We also need it to work in streamlit, without taking forever. So, there should be an updated fights_df with the NEXT fights added, which can then update the calculations on a consistant basis. 

In [15]:
def get_next_events_ufc_com(url):

    # make a looping version of get_next_events(url) 

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('div', class_='c-card-event--result__info')
    n = 0
    next_events_data = pd.DataFrame()

    for event in events:
        event_txt = events[n].text
        event_url = events[n].find('a')['href']
        event_url = 'https://www.ufc.com' + event_url
        event_title = event_txt.split('\n')[1]
        event_time = event_txt.split('/')[1]
        next_events_data = next_events_data.append({'event_title': event_title, 'event_url': event_url, 'event_date': event_time}, ignore_index=True)
        n += 1

    # only keep top 6 events
    next_events_data = next_events_data.head(6)
    return next_events_data

In [16]:
nel = get_next_events_ufc_com('https://www.ufc.com/events')
nel

,event_title,event_url,event_date
0,Lewis vs Spivac,https://www.ufc.com/event/ufc-fight-night-febr...,1:00 AM EST
1,Makhachev vs Volkanovski,https://www.ufc.com/event/ufc-284,10:00 PM EST
2,Vera vs Sandhagen,https://www.ufc.com/event/ufc-fight-night-febr...,7:00 PM EST
3,TBD vs TBD,https://www.ufc.com/event/ufc-fight-night-febr...,7:00 PM EST
4,Jones vs Gane,https://www.ufc.com/event/ufc-285,10:00 PM EST
5,Yan vs Dvalishvili,https://www.ufc.com/event/ufc-fight-night-marc...,7:00 PM EST


In [17]:
def get_fighter_urls(event_details_url):
    page = requests.get(event_details_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('tr', class_='b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click')
    n = 0
    next_event_data = pd.DataFrame()

    for event in events:
        fighters = events[n].find_all('p', class_='b-fight-details__table-text')
        fighter1 = fighters[0].text
        fighter1 = fighter1.replace('  ', '').replace('\n', '').strip()
        fighter2 = fighters[1].text
        fighter2 = fighter2.replace('  ', '').replace('\n', '').strip()
        fighter1_url = fighters[0].find('a')['href']
        fighter2_url = fighters[1].find('a')['href']
        # get weight class
        weight_class = events[n].text
        wc = weight_class.find('Matchup')
        # delete everything from the end of word "Matchup" to the begining of the weight class
        weight_class = weight_class[wc:]

        weight_class = weight_class.replace('\n', '')
        weight_class = weight_class.replace('Matchup', '')
        weight_class = weight_class.replace('  ', '').strip()
        # delete everything other then the word with "weight" in it
        #weight_class = weight_class.split(' ')
        #weight_class = [n for n in weight_class if 'weight' in n]
        next_event_data = next_event_data.append({'fighter1' :fighter1, 'fighter2:' : fighter2, 'fighter1_url': fighter1_url, 
                                                    'fighter2_url':fighter2_url, 'fight#' : n+1, 'fight_weightclass' :weight_class}, ignore_index = True)
        n += 1

    return next_event_data


In [18]:
# test
test = get_fighter_urls('http://ufcstats.com/event-details/56ec58954158966a')

In [19]:
# Get events from ufcstats

url = 'http://www.ufcstats.com/statistics/events/upcoming'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
    # get events
event = soup.findAll('i', class_ = 'b-statistics__table-content')
urls = soup.findAll('a', class_ = 'b-link b-link_style_black')
dates = soup.findAll('span', class_ = 'b-statistics__date')
n = 0
next_event_data = pd.DataFrame()

for i in event:
    # get event title, date, and url
    title = event[n].text
    title = title.replace('\n', '').strip()
    ti = title.split('  ')[0]
    date = dates[n].text.replace('\n', '').strip()
    #date = date.split('  ')[1]
    url = urls[n]['href']
    next_event_data = next_event_data.append({'event_title': ti, 'event_url': url, 'event_date': date}, ignore_index=True)
    n += 1

next_event_data= next_event_data.head(3)
next_event_data

,event_title,event_url,event_date
0,UFC Fight Night: Lewis vs. Spivac,http://www.ufcstats.com/event-details/d26394fc...,"February 04, 2023"
1,UFC 284: Makhachev vs. Volkanovski,http://www.ufcstats.com/event-details/01dd4cdc...,"February 11, 2023"
2,UFC Fight Night: Vera vs. Sandhagen,http://www.ufcstats.com/event-details/f21a3d68...,"February 18, 2023"


In [20]:
fighter_urls = get_fighter_urls(next_event_data['event_url'].values[0])
fighter_urls

,fighter1,fighter2:,fighter1_url,fighter2_url,fight#,fight_weightclass
0,Derrick Lewis,Serghei Spivac,http://www.ufcstats.com/fighter-details/d3df1a...,http://www.ufcstats.com/fighter-details/e2f6b2...,1,Heavyweight
1,Da-Un Jung,Devin Clark,http://www.ufcstats.com/fighter-details/74f944...,http://www.ufcstats.com/fighter-details/cc7040...,2,Light Heavyweight
2,Marcin Tybura,Blagoy Ivanov,http://www.ufcstats.com/fighter-details/c9cf75...,http://www.ufcstats.com/fighter-details/1788ee...,3,Heavyweight
3,Dooho Choi,Kyle Nelson,http://www.ufcstats.com/fighter-details/e93b04...,http://www.ufcstats.com/fighter-details/31f808...,4,Featherweight
4,Yusaku Kinoshita,Adam Fugitt,http://www.ufcstats.com/fighter-details/21b5c8...,http://www.ufcstats.com/fighter-details/a01a62...,5,Welterweight
5,Jeka Saragih,Anshul Jubli,http://www.ufcstats.com/fighter-details/57186c...,http://www.ufcstats.com/fighter-details/a72a2a...,6,Lightweight
6,JeongYeong Lee,Yi Zha,http://www.ufcstats.com/fighter-details/d008d7...,http://www.ufcstats.com/fighter-details/3f11fd...,7,Featherweight
7,Toshiomi Kazama,Rinya Nakamura,http://www.ufcstats.com/fighter-details/148bb1...,http://www.ufcstats.com/fighter-details/d8c811...,8,Bantamweight
8,SeungGuk Choi,HyunSung Park,http://www.ufcstats.com/fighter-details/adccbc...,http://www.ufcstats.com/fighter-details/b671bd...,9,Flyweight
9,Ji Yeon Kim,Mandy Bohm,http://www.ufcstats.com/fighter-details/a20f23...,http://www.ufcstats.com/fighter-details/297a2b...,10,Women's Flyweight


In [21]:
# turn event_date to actual date
next_event_data['event_date'] = pd.to_datetime(next_event_data['event_date']).dt.date
event_date = next_event_data['event_date'].values[0]
event_date

datetime.date(2023, 2, 4)

In [22]:
next_fight_df = fighter_urls[['fighter1', 'fighter2:', 'fight_weightclass']]
next_fight_df.columns = ['Fighter_A', 'Fighter_B', 'fight_weightclass']
next_fight_df['date'] = event_date
next_fight_df

,Fighter_A,Fighter_B,fight_weightclass,date
0,Derrick Lewis,Serghei Spivac,Heavyweight,2023-02-04
1,Da-Un Jung,Devin Clark,Light Heavyweight,2023-02-04
2,Marcin Tybura,Blagoy Ivanov,Heavyweight,2023-02-04
3,Dooho Choi,Kyle Nelson,Featherweight,2023-02-04
4,Yusaku Kinoshita,Adam Fugitt,Welterweight,2023-02-04
5,Jeka Saragih,Anshul Jubli,Lightweight,2023-02-04
6,JeongYeong Lee,Yi Zha,Featherweight,2023-02-04
7,Toshiomi Kazama,Rinya Nakamura,Bantamweight,2023-02-04
8,SeungGuk Choi,HyunSung Park,Flyweight,2023-02-04
9,Ji Yeon Kim,Mandy Bohm,Women's Flyweight,2023-02-04


#### Get Columns to Add Features



In [23]:
# load final df to get columns from 

fights = pd.read_csv('data/final/aggregates/Double_Fights_DF_V14.csv') 

In [24]:
fights.columns.to_list()

['Fighter_A',
 'Fighter_B',
 'A_Kd',
 'B_Kd',
 'A_Sig_strike_land',
 'A_Sig_strike_att',
 'B_Sig_strike_land',
 'B_Sig_strike_att',
 'A_Sig_strike_percent',
 'B_Sig_strike_percent',
 'A_Total_Strikes_land',
 'A_Total_Strikes_att',
 'B_Total_Strikes_land',
 'B_Total_Strikes_att',
 'A_Total_Strikes_percent',
 'B_Total_Strikes_percent',
 'A_Takedowns_land',
 'A_Takedowns_att',
 'B_Takedowns_land',
 'B_Takedowns_att',
 'A_Takedown_percent',
 'B_Takedown_percent',
 'A_Sub_Attempts_land',
 'A_Sub_Attempts_att',
 'B_Sub_Attempts_land',
 'B_Sub_Attempts_att',
 'A_Rev',
 'B_Rev',
 'A_Ctrl_time_min',
 'A_Ctrl_time_sec',
 'B_Ctrl_time_min',
 'B_Ctrl_time_sec',
 'A_Ctrl_time_tot',
 'B_Ctrl_time_tot',
 'details',
 'event_title',
 'event_url',
 'date',
 'Winner',
 'fight_id',
 'Unnamed: 0',
 'A_Head_Strikes_land',
 'A_Head_Strikes_att',
 'B_Head_Strikes_land',
 'B_Head_Strikes_att',
 'A_Head_Strikes_percent',
 'B_Head_Strikes_percent',
 'A_Body_Strikes_land',
 'A_Body_Strikes_att',
 'B_Body_Strikes_

In [25]:
fights.columns

Index(['Fighter_A', 'Fighter_B', 'A_Kd', 'B_Kd', 'A_Sig_strike_land',
       'A_Sig_strike_att', 'B_Sig_strike_land', 'B_Sig_strike_att',
       'A_Sig_strike_percent', 'B_Sig_strike_percent',
       ...
       'Dif_Opp_Avg_Body_Strikes_land_per_round',
       'Dif_Opp_Avg_Body_Strikes_att_per_round',
       'Dif_Opp_Avg_Leg_Strikes_land_per_round',
       'Dif_Opp_Avg_Leg_Strikes_att_per_round',
       'Dif_Opp_Avg_Distance_Strikes_land_per_round',
       'Dif_Opp_Avg_Distance_Strikes_att_per_round',
       'Dif_Opp_Avg_Clinch_Strikes_land_per_round',
       'Dif_Opp_Avg_Clinch_Strikes_att_per_round',
       'Dif_Opp_Avg_Ground_Strikes_land_per_round',
       'Dif_Opp_Avg_Ground_Strikes_att_per_round'],
      dtype='object', length=783)

### Add Features

- Start with Fighter Names
- Add odds data


In [26]:
next_fight_df

,Fighter_A,Fighter_B,fight_weightclass,date
0,Derrick Lewis,Serghei Spivac,Heavyweight,2023-02-04
1,Da-Un Jung,Devin Clark,Light Heavyweight,2023-02-04
2,Marcin Tybura,Blagoy Ivanov,Heavyweight,2023-02-04
3,Dooho Choi,Kyle Nelson,Featherweight,2023-02-04
4,Yusaku Kinoshita,Adam Fugitt,Welterweight,2023-02-04
5,Jeka Saragih,Anshul Jubli,Lightweight,2023-02-04
6,JeongYeong Lee,Yi Zha,Featherweight,2023-02-04
7,Toshiomi Kazama,Rinya Nakamura,Bantamweight,2023-02-04
8,SeungGuk Choi,HyunSung Park,Flyweight,2023-02-04
9,Ji Yeon Kim,Mandy Bohm,Women's Flyweight,2023-02-04


In [27]:
def get_odds(url):
    
        # to scrape odds from BFO
        # you can scrape by pretending to be a browser

        header = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
        }
        r = requests.get(url, headers = header)
        dfs = pd.read_html(r.text)
        data = dfs[1]
        data.rename(columns={'Unnamed: 0': 'fighter'}, inplace=True)
        data['event_odds_url'] = url
        event_name = url.split('/')[4]
        data['event_id'] = event_name
        data.to_csv('data/final/events/odds_per_event/' + event_name + '.csv', index=False)
        return data

In [28]:
event_odds = get_odds(next['bfo_url'].values[0])

In [29]:
event_odds = event_odds[['fighter', 'DraftKings', '5D', 'Ref']]
event_odds

,fighter,DraftKings,5D,Ref
0,Derrick Lewis,+190▼,+185▼,+190▼
1,Serghei Spivac,-225▲,-225▲,-225▲
2,Over 1½ rounds,NaN,+100▲,-105▲
3,Under 1½ rounds,NaN,-120▼,-125▼
4,Over 2½ rounds,NaN,+195▲,NaN
...,...,...,...,...
181,Not Taira inside distance,NaN,+100,NaN
182,Aguilar wins by decision,NaN,+1500,NaN
183,Not Aguilar by decision,NaN,-4500,NaN
184,Taira wins by decision,NaN,+160,NaN


In [30]:
# get rid of arrows in all columns
event_odds['DraftKings'] = event_odds['DraftKings'].str.replace('▲', '')
event_odds['DraftKings'] = event_odds['DraftKings'].str.replace('▼', '')
event_odds['5D'] = event_odds['5D'].str.replace('▲', '')
event_odds['5D'] = event_odds['5D'].str.replace('▼', '')
event_odds['Ref'] = event_odds['Ref'].str.replace('▲', '')
event_odds['Ref'] = event_odds['Ref'].str.replace('▼', '')

In [31]:
# turn to numeric
event_odds['DraftKings'] = pd.to_numeric(event_odds['DraftKings'])
event_odds['5D'] = pd.to_numeric(event_odds['5D'])
event_odds['Ref'] = pd.to_numeric(event_odds['Ref'])

In [32]:
def get_odds_for_fighter(fighter):
    try:
        data = event_odds[event_odds['fighter'] == fighter]
        dk_odds = data['DraftKings'].values[0]
        return dk_odds
    except:
        return np.nan


In [33]:
next_fight_df['Fighter_A_Odds'] = next_fight_df['Fighter_A'].apply(get_odds_for_fighter)
next_fight_df['Fighter_B_Odds'] = next_fight_df['Fighter_B'].apply(get_odds_for_fighter)

In [34]:
next_fight_df

,Fighter_A,Fighter_B,fight_weightclass,date,Fighter_A_Odds,Fighter_B_Odds
0,Derrick Lewis,Serghei Spivac,Heavyweight,2023-02-04,190.0,-225.0
1,Da-Un Jung,Devin Clark,Light Heavyweight,2023-02-04,NaN,205.0
2,Marcin Tybura,Blagoy Ivanov,Heavyweight,2023-02-04,-190.0,160.0
3,Dooho Choi,Kyle Nelson,Featherweight,2023-02-04,NaN,180.0
4,Yusaku Kinoshita,Adam Fugitt,Welterweight,2023-02-04,-265.0,225.0
5,Jeka Saragih,Anshul Jubli,Lightweight,2023-02-04,-105.0,-115.0
6,JeongYeong Lee,Yi Zha,Featherweight,2023-02-04,NaN,210.0
7,Toshiomi Kazama,Rinya Nakamura,Bantamweight,2023-02-04,330.0,-410.0
8,SeungGuk Choi,HyunSung Park,Flyweight,2023-02-04,NaN,NaN
9,Ji Yeon Kim,Mandy Bohm,Women's Flyweight,2023-02-04,-285.0,240.0


#### Add Odds Change

In [35]:
def get_bfodds(event_url):
    # scrape the changes in odds by event url
    
    driver = webdriver.Chrome()
    driver.get(event_url)

    # click show all
    driver.find_element_by_class_name('event-swing-expand').click()
    #wait 2 seconds
    time.sleep(2)

    event_swing_container = driver.find_element_by_id('event-swing-container')
    event_swing_container.text


    # the first half of the text is the odds changes, the seconds half are the names. Lets match them up. 
    # first split the text by new line
    event_swing_container.text.split('\n')

    # get the names. You can identify them because they will be more than 5 characters long
    names = [x for x in event_swing_container.text.split('\n') if len(x) > 5]
    names

    # get the odds changes
    odds_changes = [x for x in event_swing_container.text.split('\n') if '%' in x]
    odds_changes

    # get the event name
    event_name = driver.find_element_by_class_name('table-header')
    # name is h1 in event_name
    name = event_name.find_element_by_tag_name('h1')
    event = name.text

    # get the url
    url = driver.current_url


    # create a dataframe
    df = pd.DataFrame({'names': names, 'odds_changes': odds_changes, 'event': event, 'event_url': url})
    
    #close the driver
    driver.close()

    return df

In [36]:
odds_changes = get_bfodds(next['bfo_url'].values[0])
odds_changes.head(2)

,names,odds_changes,event,event_url
0,Anshul Jubli,-43%,UFC FIGHT NIGHT: LEWIS VS. SPIVAK,https://www.bestfightodds.com/events/ufc-fight...
1,Serghei Spivac,-21%,UFC FIGHT NIGHT: LEWIS VS. SPIVAK,https://www.bestfightodds.com/events/ufc-fight...


In [37]:
# if ':' in odds_changes['odds_changes'].values[0]:
# split the odds changes by : and keep the second part
def fix_odds_changes(odds):
    if ':' in odds:
        oddz =  odds.split(':')[1].strip()
        # remove the % sign
        oddz = oddz.replace('%', '')
        oddz = float(oddz)
        return oddz
    else:
        oddz = odds.replace('%', '')
        oddz = float(oddz)
        return oddz

In [38]:
def fix_names(name):
    if "Change since opening" in name:
        return name.split('Change since opening')[0].strip()[:-1]
    else:
        return name

In [39]:
# fix the odds changes
odds_changes['odds_changes'] = odds_changes['odds_changes'].apply(fix_odds_changes)

In [40]:
odds_changes['names'] = odds_changes['names'].apply(fix_names)

In [41]:
odds_changes

,names,odds_changes,event,event_url
0,Anshul Jubli,-43.0,UFC FIGHT NIGHT: LEWIS VS. SPIVAK,https://www.bestfightodds.com/events/ufc-fight...
1,Serghei Spivac,-21.0,UFC FIGHT NIGHT: LEWIS VS. SPIVAK,https://www.bestfightodds.com/events/ufc-fight...
2,Kyle Nelson,-21.0,UFC FIGHT NIGHT: LEWIS VS. SPIVAK,https://www.bestfightodds.com/events/ufc-fight...
3,Ji Yeon Kim,-19.0,UFC FIGHT NIGHT: LEWIS VS. SPIVAK,https://www.bestfightodds.com/events/ufc-fight...
4,Lee Jeong-Yeong,-16.0,UFC FIGHT NIGHT: LEWIS VS. SPIVAK,https://www.bestfightodds.com/events/ufc-fight...
5,Marcin Tybura,-13.0,UFC FIGHT NIGHT: LEWIS VS. SPIVAK,https://www.bestfightodds.com/events/ufc-fight...
6,Blagoy Ivanov,-12.0,UFC FIGHT NIGHT: LEWIS VS. SPIVAK,https://www.bestfightodds.com/events/ufc-fight...
7,Seung Guk Choi,-11.0,UFC FIGHT NIGHT: LEWIS VS. SPIVAK,https://www.bestfightodds.com/events/ufc-fight...
8,Yi Zha,-10.0,UFC FIGHT NIGHT: LEWIS VS. SPIVAK,https://www.bestfightodds.com/events/ufc-fight...
9,Jun Yong Park,-7.0,UFC FIGHT NIGHT: LEWIS VS. SPIVAK,https://www.bestfightodds.com/events/ufc-fight...


In [42]:
def get_odds_changes_for_fighter(fighter):
    try:
        data = odds_changes[odds_changes['names'] == fighter]
        odds_change = data['odds_changes'].values[0]
        return odds_change
    except:
        return np.nan

In [43]:
next_fight_df['Fighter_A_Odds_Change'] = next_fight_df['Fighter_A'].apply(get_odds_changes_for_fighter)
next_fight_df['Fighter_B_Odds_Change'] = next_fight_df['Fighter_B'].apply(get_odds_changes_for_fighter)


In [44]:
next_fight_df

,Fighter_A,Fighter_B,fight_weightclass,date,Fighter_A_Odds,Fighter_B_Odds,Fighter_A_Odds_Change,Fighter_B_Odds_Change
0,Derrick Lewis,Serghei Spivac,Heavyweight,2023-02-04,190.0,-225.0,26.0,-21.0
1,Da-Un Jung,Devin Clark,Light Heavyweight,2023-02-04,NaN,205.0,NaN,23.0
2,Marcin Tybura,Blagoy Ivanov,Heavyweight,2023-02-04,-190.0,160.0,-13.0,-12.0
3,Dooho Choi,Kyle Nelson,Featherweight,2023-02-04,NaN,180.0,NaN,-21.0
4,Yusaku Kinoshita,Adam Fugitt,Welterweight,2023-02-04,-265.0,225.0,15.0,24.0
5,Jeka Saragih,Anshul Jubli,Lightweight,2023-02-04,-105.0,-115.0,73.0,-43.0
6,JeongYeong Lee,Yi Zha,Featherweight,2023-02-04,NaN,210.0,NaN,-10.0
7,Toshiomi Kazama,Rinya Nakamura,Bantamweight,2023-02-04,330.0,-410.0,32.0,6.0
8,SeungGuk Choi,HyunSung Park,Flyweight,2023-02-04,NaN,NaN,NaN,NaN
9,Ji Yeon Kim,Mandy Bohm,Women's Flyweight,2023-02-04,-285.0,240.0,-19.0,24.0


### Add Distribution Stat Features

In [45]:
# for each column in all_metric_cols, get the mean, std, etc. for each fighter
def get_fighter_running_dist_stats(fighter, date, col_to_get, stat_to_calc):
    data = fights[(fights['Fighter_A'] == fighter) | (fights['Fighter_B'] == fighter)]

    # only get fights before the date
    datey = pd.to_datetime(date)
    data['date'] = pd.to_datetime(data['date'])
    data = data[data['date'] < datey]

    # fighter could be either fighter A or fighter B
    fighter_data = pd.DataFrame()
    # when fighter is fighter A, get all fighter A data and copy it to fighter_data

    # fighterA df
    fighterA_df = fights[fights['Fighter_A'] == fighter]
    fighterB_df = fights[fights['Fighter_B'] == fighter]

    # keep only the fighters columns date, FighterA, and the col_to_get, 
    # do same for B, change col names from B to A, and concat
    fighterA_df = fighterA_df[['date', 'Fighter_A', 'A_' + col_to_get]]
    fighterA_df.rename(columns={'A_' + col_to_get: col_to_get, 'Fighter_A': 'fighter'}, inplace=True)
    fighterB_df = fighterB_df[['date', 'Fighter_B', 'B_' + col_to_get]]
    fighterB_df.rename(columns={'B_' + col_to_get: col_to_get, 'Fighter_B': 'fighter'}, inplace=True)

    # append the dataframes on fighter
    fighter_data = fighter_data.append(fighterA_df)
    fighter_data = fighter_data.append(fighterB_df)

    # get the chosen statistic
    if stat_to_calc == 'mean':
        x = fighter_data[col_to_get].mean()
    elif stat_to_calc == 'std':
        x = fighter_data[col_to_get].std()
    elif stat_to_calc == 'median':
        x = fighter_data[col_to_get].median()  
    return x

In [46]:
# get non-specific column names for feature creation

dif_cols = [n for n in fights.columns if 'Dif_' in n]
A_cols = [n for n in fights.columns if n.startswith('A_')]
all_metric_cols = A_cols + dif_cols
A_cols2 = pd.DataFrame(A_cols)
A_cols2['nonspecific'] = A_cols2[0].str[2:]
the_cols = list(A_cols2['nonspecific'].unique())
the_cols = the_cols[:34]
the_cols

['Kd',
 'Sig_strike_land',
 'Sig_strike_att',
 'Sig_strike_percent',
 'Total_Strikes_land',
 'Total_Strikes_att',
 'Total_Strikes_percent',
 'Takedowns_land',
 'Takedowns_att',
 'Takedown_percent',
 'Sub_Attempts_land',
 'Sub_Attempts_att',
 'Rev',
 'Ctrl_time_min',
 'Ctrl_time_sec',
 'Ctrl_time_tot',
 'Head_Strikes_land',
 'Head_Strikes_att',
 'Head_Strikes_percent',
 'Body_Strikes_land',
 'Body_Strikes_att',
 'Body_Strikes_percent',
 'Leg_Strikes_land',
 'Leg_Strikes_att',
 'Leg_Strikes_percent',
 'Distance_Strikes_land',
 'Distance_Strikes_att',
 'Distance_Strikes_percent',
 'Clinch_Strikes_land',
 'Clinch_Strikes_att',
 'Clinch_Strikes_percent',
 'Ground_Strikes_land',
 'Ground_Strikes_att',
 'Ground_Strikes_percent']

In [47]:
for col in the_cols:
    for stat in ['mean', 'std', 'median']:
        next_fight_df['A_Rolling_' + col + '_' + stat] = next_fight_df.apply(lambda row: get_fighter_running_dist_stats(row['Fighter_A'], row['date'], col, stat), axis=1)
        next_fight_df['B_Rolling_' + col + '_' + stat] = next_fight_df.apply(lambda row: get_fighter_running_dist_stats(row['Fighter_B'], row['date'], col, stat), axis=1)

In [48]:
fights

,Fighter_A,Fighter_B,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,B_Sig_strike_percent,...,Dif_Opp_Avg_Body_Strikes_land_per_round,Dif_Opp_Avg_Body_Strikes_att_per_round,Dif_Opp_Avg_Leg_Strikes_land_per_round,Dif_Opp_Avg_Leg_Strikes_att_per_round,Dif_Opp_Avg_Distance_Strikes_land_per_round,Dif_Opp_Avg_Distance_Strikes_att_per_round,Dif_Opp_Avg_Clinch_Strikes_land_per_round,Dif_Opp_Avg_Clinch_Strikes_att_per_round,Dif_Opp_Avg_Ground_Strikes_land_per_round,Dif_Opp_Avg_Ground_Strikes_att_per_round
0,Holly Holm,Irene Aldana,0,0,154,301,69,185,0.51,0.37,...,-0.438261,-0.773091,-0.375092,-0.354364,-2.452268,-6.012022,0.171166,0.113466,-0.117277,-0.271253
1,Greg Hardy,Ben Sosoli,0,0,54,105,26,97,0.51,0.26,...,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,NaN,NaN
2,Jared Rosholt,Josh Copeland,0,0,22,45,9,36,0.48,0.25,...,-3.222222,-3.944444,-1.888889,-2.194444,-7.500000,-19.138889,-1.888889,-1.694444,-1.138889,-2.166667
3,Alistair Overeem,Stefan Struve,0,0,17,25,2,5,0.68,0.40,...,1.448693,1.796830,0.142988,0.268794,3.504557,9.869592,2.512546,3.503052,0.838169,0.800327
4,Nordine Taleb,Kyle Prepolec,0,0,90,201,52,153,0.44,0.33,...,-inf,-inf,-inf,-inf,-inf,-inf,NaN,NaN,-inf,-inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8133,Charles Oliveira,Nik Lentz,1,0,64,84,37,69,0.76,0.53,...,0.101463,0.196173,0.033065,0.082716,0.238758,0.651701,-0.044744,-0.044639,0.249784,0.458868
8134,Marcus Davis,Mike Swick,0,0,31,55,32,86,0.56,0.37,...,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
8135,Lucas Martins,Darren Elkins,0,0,26,82,47,154,0.31,0.30,...,2.014588,3.750604,1.567838,1.972714,6.914827,17.731710,2.203702,4.730845,2.377590,3.994237
8136,Krzysztof Jotko,Sean Strickland,0,0,37,196,84,182,0.18,0.46,...,-0.503041,-1.072637,-0.311983,-0.447958,-1.430133,-4.644413,0.129017,0.149822,-0.078125,-0.127501


### Features: Top-Down Averages (Typical UFC Style)

In [49]:
def get_top_down_averages(fighter, date, col_to_get, dataframe):

    # for each column in all_metric_cols, calculate the top-down averages, as opposed to bottom-up

    data = dataframe[(dataframe['Fighter_A'] == fighter) | (dataframe['Fighter_B'] == fighter)]

    # only get fights before the date

    datey = pd.to_datetime(date)
    data['date'] = pd.to_datetime(data['date'])
    data = data[data['date'] < datey]

    # fighter could be either fighter A or fighter B

    fighter_data = pd.DataFrame()

    # when fighter is fighter A, get all fighter A data and copy it to fighter_data

    # fighterA df

    fighterA_df = dataframe[dataframe['Fighter_A'] == fighter]
    fighterB_df = dataframe[dataframe['Fighter_B'] == fighter]
    
    # keep only the fighters columns date, FighterA, and the col_to_get, 
    # do same for B, change col names from B to A, and concat

    fighterA_df = fighterA_df[['date', 'Fighter_A', 'A_' + col_to_get]]
    fighterA_df.rename(columns={'A_' + col_to_get: col_to_get, 'Fighter_A': 'fighter'}, inplace=True)
    fighterB_df = fighterB_df[['date', 'Fighter_B', 'B_' + col_to_get]]
    fighterB_df.rename(columns={'B_' + col_to_get: col_to_get, 'Fighter_B': 'fighter'}, inplace=True)
    fighter_data = fighter_data.append(fighterA_df)
    fighter_data = fighter_data.append(fighterB_df)

    # append the dataframes on fighter

    fighter_data = fighter_data.append(fighterA_df)
    fighter_data = fighter_data.append(fighterB_df)

    # get the average

    tot = fighter_data[col_to_get].sum()
    num = fighter_data[col_to_get].count()
    x = tot / num
    return x

In [50]:
# NEW

for col in the_cols:
    next_fight_df['A_topdown_Avg_' + col ] = next_fight_df.apply(lambda row: get_top_down_averages(row['Fighter_A'], row['date'], col, fights), axis=1)
    next_fight_df['B_topdown_Avg_' + col ] = next_fight_df.apply(lambda row: get_top_down_averages(row['Fighter_B'], row['date'], col, fights), axis=1)

### Add Opponent Stats

In [51]:
def get_opponent_averages(datafrm, date, col_to_get, fighter):
    col_to_get = 'B_' + col_to_get
    data = datafrm[(datafrm['Fighter_A'] == fighter) | (datafrm['Fighter_B'] == fighter)]
    datey = pd.to_datetime(date)
    data['date'] = pd.to_datetime(data['date'])
    data = data[data['date'] < datey]

    fighter_data = pd.DataFrame()

    fighterA_df = datafrm[datafrm['Fighter_A'] == fighter]
    fighterB_df = datafrm[datafrm['Fighter_B'] == fighter]

    # switch A_ and B_ to B_ and A_ in fighterB_df
    fighterB_df.columns = [n.replace('A_', 'C_') if n.startswith('A_') else n for n in fighterB_df.columns]
    fighterB_df.columns = [n.replace('B_', 'D_') if n.startswith('B_') else n for n in fighterB_df.columns]
    fighterB_df.columns = [n.replace('C_', 'B_') if n.startswith('C_') else n for n in fighterB_df.columns]
    fighterB_df.columns = [n.replace('D_', 'A_') if n.startswith('D_') else n for n in fighterB_df.columns]

    # copy fighter_A and fighter_B columns to new dataframe, then switch fighter_A and fighter_B columns

    fighterB_df.rename(columns={'Fighter_A': 'Fighter_A2', 'Fighter_B': 'Fighter_B2'}, inplace=True)
    fighterB_df.rename(columns={'Fighter_B2': 'Fighter_A', 'Fighter_A2': 'Fighter_B'}, inplace=True)

    fighter_data = fighter_data.append(fighterA_df)
    fighter_data = fighter_data.append(fighterB_df)

    # get sum of col to get
    col_sum = fighter_data[col_to_get].sum()
    # get number of fights
    num_fights = len(fighter_data)
    # get average
    avg = col_sum / num_fights

    return avg


In [52]:
n=0

for col in the_cols:
    next_fight_df['A_Opp_Avg_' + col] = next_fight_df.apply(lambda row: get_opponent_averages(fights, row['date'], col, row['Fighter_A']), axis=1)
    next_fight_df['B_Opp_Avg_' + col] = next_fight_df.apply(lambda row: get_opponent_averages(fights, row['date'], col, row['Fighter_B']), axis=1)
    n = n+1
    print(f' {col}, #{n} / {len(the_cols)} done.')

 Kd, #1 / 34 done.
 Sig_strike_land, #2 / 34 done.
 Sig_strike_att, #3 / 34 done.
 Sig_strike_percent, #4 / 34 done.
 Total_Strikes_land, #5 / 34 done.
 Total_Strikes_att, #6 / 34 done.
 Total_Strikes_percent, #7 / 34 done.
 Takedowns_land, #8 / 34 done.
 Takedowns_att, #9 / 34 done.
 Takedown_percent, #10 / 34 done.
 Sub_Attempts_land, #11 / 34 done.
 Sub_Attempts_att, #12 / 34 done.
 Rev, #13 / 34 done.
 Ctrl_time_min, #14 / 34 done.
 Ctrl_time_sec, #15 / 34 done.
 Ctrl_time_tot, #16 / 34 done.
 Head_Strikes_land, #17 / 34 done.
 Head_Strikes_att, #18 / 34 done.
 Head_Strikes_percent, #19 / 34 done.
 Body_Strikes_land, #20 / 34 done.
 Body_Strikes_att, #21 / 34 done.
 Body_Strikes_percent, #22 / 34 done.
 Leg_Strikes_land, #23 / 34 done.
 Leg_Strikes_att, #24 / 34 done.
 Leg_Strikes_percent, #25 / 34 done.
 Distance_Strikes_land, #26 / 34 done.
 Distance_Strikes_att, #27 / 34 done.
 Distance_Strikes_percent, #28 / 34 done.
 Clinch_Strikes_land, #29 / 34 done.
 Clinch_Strikes_att, #30

In [53]:
next_fight_df

,Fighter_A,Fighter_B,fight_weightclass,date,Fighter_A_Odds,Fighter_B_Odds,Fighter_A_Odds_Change,Fighter_B_Odds_Change,A_Rolling_Kd_mean,B_Rolling_Kd_mean,...,A_Opp_Avg_Clinch_Strikes_att,B_Opp_Avg_Clinch_Strikes_att,A_Opp_Avg_Clinch_Strikes_percent,B_Opp_Avg_Clinch_Strikes_percent,A_Opp_Avg_Ground_Strikes_land,B_Opp_Avg_Ground_Strikes_land,A_Opp_Avg_Ground_Strikes_att,B_Opp_Avg_Ground_Strikes_att,A_Opp_Avg_Ground_Strikes_percent,B_Opp_Avg_Ground_Strikes_percent
0,Derrick Lewis,Serghei Spivac,Heavyweight,2023-02-04,190.0,-225.0,26.0,-21.0,0.363636,0.00,...,6.090909,2.00,0.506032,0.333333,4.727273,0.00,6.090909,0.00,0.447090,0.000000
1,Da-Un Jung,Devin Clark,Light Heavyweight,2023-02-04,NaN,205.0,NaN,23.0,NaN,0.20,...,NaN,14.80,NaN,0.731615,NaN,5.60,NaN,6.90,NaN,0.483636
2,Marcin Tybura,Blagoy Ivanov,Heavyweight,2023-02-04,-190.0,160.0,-13.0,-12.0,0.071429,0.25,...,10.857143,18.75,0.726625,0.811924,2.142857,0.50,2.857143,0.50,0.336735,0.250000
3,Dooho Choi,Kyle Nelson,Featherweight,2023-02-04,NaN,180.0,NaN,-21.0,1.000000,0.00,...,4.000000,8.00,1.000000,0.842308,8.000000,12.75,10.000000,21.25,0.800000,0.553467
4,Yusaku Kinoshita,Adam Fugitt,Welterweight,2023-02-04,-265.0,225.0,15.0,24.0,NaN,0.00,...,NaN,17.00,NaN,0.705882,NaN,7.00,NaN,7.00,NaN,1.000000
5,Jeka Saragih,Anshul Jubli,Lightweight,2023-02-04,-105.0,-115.0,73.0,-43.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,JeongYeong Lee,Yi Zha,Featherweight,2023-02-04,NaN,210.0,NaN,-10.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Toshiomi Kazama,Rinya Nakamura,Bantamweight,2023-02-04,330.0,-410.0,32.0,6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SeungGuk Choi,HyunSung Park,Flyweight,2023-02-04,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Ji Yeon Kim,Mandy Bohm,Women's Flyweight,2023-02-04,-285.0,240.0,-19.0,24.0,NaN,0.00,...,NaN,19.00,NaN,0.759420,NaN,4.50,NaN,7.50,NaN,0.785714


### Feature: Rolling Career Stat Differentials

#### 1) Mean (Average)

In [54]:
rolling_mean_cols= [n for n in fights.columns if 'Rolling' in n and 'mean' in n]
# no difs
rolling_mean_cols= [n for n in rolling_mean_cols if 'Dif' not in n]
rolling_mean_cols = pd.DataFrame(rolling_mean_cols)
rolling_mean_cols['nonspecific'] = rolling_mean_cols[0].str[2:]
rmc = rolling_mean_cols['nonspecific'].unique()
rmc

array(['Rolling_Kd_mean', 'Rolling_Sig_strike_land_mean',
       'Rolling_Sig_strike_att_mean', 'Rolling_Sig_strike_percent_mean',
       'Rolling_Total_Strikes_land_mean',
       'Rolling_Total_Strikes_att_mean',
       'Rolling_Total_Strikes_percent_mean',
       'Rolling_Takedowns_land_mean', 'Rolling_Takedowns_att_mean',
       'Rolling_Takedown_percent_mean', 'Rolling_Sub_Attempts_land_mean',
       'Rolling_Sub_Attempts_att_mean', 'Rolling_Rev_mean',
       'Rolling_Ctrl_time_min_mean', 'Rolling_Ctrl_time_sec_mean',
       'Rolling_Ctrl_time_tot_mean', 'Rolling_Head_Strikes_land_mean',
       'Rolling_Head_Strikes_att_mean',
       'Rolling_Head_Strikes_percent_mean',
       'Rolling_Body_Strikes_land_mean', 'Rolling_Body_Strikes_att_mean',
       'Rolling_Body_Strikes_percent_mean',
       'Rolling_Leg_Strikes_land_mean', 'Rolling_Leg_Strikes_att_mean',
       'Rolling_Leg_Strikes_percent_mean',
       'Rolling_Distance_Strikes_land_mean',
       'Rolling_Distance_Strikes_att_me

In [55]:
for col in rmc:
    next_fight_df['Dif_' + col] = next_fight_df['A_' + col] - next_fight_df['B_' + col]

next_fight_df.head(3)

,Fighter_A,Fighter_B,fight_weightclass,date,Fighter_A_Odds,Fighter_B_Odds,Fighter_A_Odds_Change,Fighter_B_Odds_Change,A_Rolling_Kd_mean,B_Rolling_Kd_mean,...,Dif_Rolling_Leg_Strikes_percent_mean,Dif_Rolling_Distance_Strikes_land_mean,Dif_Rolling_Distance_Strikes_att_mean,Dif_Rolling_Distance_Strikes_percent_mean,Dif_Rolling_Clinch_Strikes_land_mean,Dif_Rolling_Clinch_Strikes_att_mean,Dif_Rolling_Clinch_Strikes_percent_mean,Dif_Rolling_Ground_Strikes_land_mean,Dif_Rolling_Ground_Strikes_att_mean,Dif_Rolling_Ground_Strikes_percent_mean
0,Derrick Lewis,Serghei Spivac,Heavyweight,2023-02-04,190.0,-225.0,26.0,-21.0,0.363636,0.00,...,0.538258,-8.121212,-18.984848,0.084780,3.727273,6.000000,0.238059,-10.833333,-13.106061,-0.404883
1,Da-Un Jung,Devin Clark,Light Heavyweight,2023-02-04,NaN,205.0,NaN,23.0,NaN,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Marcin Tybura,Blagoy Ivanov,Heavyweight,2023-02-04,-190.0,160.0,-13.0,-12.0,0.071429,0.25,...,-0.248816,-18.142857,-34.392857,-0.049023,-1.678571,-1.928571,-0.262116,7.357143,10.607143,0.025065


#### 2) Median (Average)

In [56]:
rolling_median_cols= [n for n in fights.columns if 'Rolling' in n and 'median' in n]
# no difs
rolling_median_cols= [n for n in rolling_median_cols if 'Dif' not in n]
rolling_median_cols = pd.DataFrame(rolling_median_cols)
rolling_median_cols['nonspecific'] = rolling_median_cols[0].str[2:]
rmc = rolling_median_cols['nonspecific'].unique()

In [57]:
for col in rmc:
    next_fight_df['Dif_' + col] = next_fight_df['A_' + col] - next_fight_df['B_' + col]

next_fight_df.head(3)

,Fighter_A,Fighter_B,fight_weightclass,date,Fighter_A_Odds,Fighter_B_Odds,Fighter_A_Odds_Change,Fighter_B_Odds_Change,A_Rolling_Kd_mean,B_Rolling_Kd_mean,...,Dif_Rolling_Leg_Strikes_percent_median,Dif_Rolling_Distance_Strikes_land_median,Dif_Rolling_Distance_Strikes_att_median,Dif_Rolling_Distance_Strikes_percent_median,Dif_Rolling_Clinch_Strikes_land_median,Dif_Rolling_Clinch_Strikes_att_median,Dif_Rolling_Clinch_Strikes_percent_median,Dif_Rolling_Ground_Strikes_land_median,Dif_Rolling_Ground_Strikes_att_median,Dif_Rolling_Ground_Strikes_percent_median
0,Derrick Lewis,Serghei Spivac,Heavyweight,2023-02-04,190.0,-225.0,26.0,-21.0,0.363636,0.00,...,0.733333,-1.5,5.5,-0.027950,2.5,4.0,0.666667,-22.5,-24.0,-0.455882
1,Da-Un Jung,Devin Clark,Light Heavyweight,2023-02-04,NaN,205.0,NaN,23.0,NaN,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Marcin Tybura,Blagoy Ivanov,Heavyweight,2023-02-04,-190.0,160.0,-13.0,-12.0,0.071429,0.25,...,-0.145378,-40.5,-58.5,-0.065125,-3.0,-5.0,-0.260606,0.5,1.0,0.156410


#### 3) Standard Deviations

In [58]:
rolling_std_cols= [n for n in fights.columns if 'Rolling' in n and 'std' in n]
# no difs
rolling_std_cols= [n for n in rolling_std_cols if 'Dif' not in n]

rolling_std_cols = pd.DataFrame(rolling_std_cols)
rolling_std_cols['nonspecific'] = rolling_std_cols[0].str[2:]
rsc = rolling_std_cols['nonspecific'].unique()
rsc

array(['Rolling_Kd_std', 'Rolling_Sig_strike_land_std',
       'Rolling_Sig_strike_att_std', 'Rolling_Sig_strike_percent_std',
       'Rolling_Total_Strikes_land_std', 'Rolling_Total_Strikes_att_std',
       'Rolling_Total_Strikes_percent_std', 'Rolling_Takedowns_land_std',
       'Rolling_Takedowns_att_std', 'Rolling_Takedown_percent_std',
       'Rolling_Sub_Attempts_land_std', 'Rolling_Sub_Attempts_att_std',
       'Rolling_Rev_std', 'Rolling_Ctrl_time_min_std',
       'Rolling_Ctrl_time_sec_std', 'Rolling_Ctrl_time_tot_std',
       'Rolling_Head_Strikes_land_std', 'Rolling_Head_Strikes_att_std',
       'Rolling_Head_Strikes_percent_std',
       'Rolling_Body_Strikes_land_std', 'Rolling_Body_Strikes_att_std',
       'Rolling_Body_Strikes_percent_std', 'Rolling_Leg_Strikes_land_std',
       'Rolling_Leg_Strikes_att_std', 'Rolling_Leg_Strikes_percent_std',
       'Rolling_Distance_Strikes_land_std',
       'Rolling_Distance_Strikes_att_std',
       'Rolling_Distance_Strikes_percent_st

In [59]:
for col in rsc:
    next_fight_df['Dif_' + col] = next_fight_df['A_' + col] - next_fight_df['B_' + col]

next_fight_df.head(3)

,Fighter_A,Fighter_B,fight_weightclass,date,Fighter_A_Odds,Fighter_B_Odds,Fighter_A_Odds_Change,Fighter_B_Odds_Change,A_Rolling_Kd_mean,B_Rolling_Kd_mean,...,Dif_Rolling_Leg_Strikes_percent_std,Dif_Rolling_Distance_Strikes_land_std,Dif_Rolling_Distance_Strikes_att_std,Dif_Rolling_Distance_Strikes_percent_std,Dif_Rolling_Clinch_Strikes_land_std,Dif_Rolling_Clinch_Strikes_att_std,Dif_Rolling_Clinch_Strikes_percent_std,Dif_Rolling_Ground_Strikes_land_std,Dif_Rolling_Ground_Strikes_att_std,Dif_Rolling_Ground_Strikes_percent_std
0,Derrick Lewis,Serghei Spivac,Heavyweight,2023-02-04,190.0,-225.0,26.0,-21.0,0.363636,0.00,...,0.475013,-11.613101,-28.465807,-0.074054,4.650445,8.220242,-0.135393,8.297361,12.645049,0.276896
1,Da-Un Jung,Devin Clark,Light Heavyweight,2023-02-04,NaN,205.0,NaN,23.0,NaN,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Marcin Tybura,Blagoy Ivanov,Heavyweight,2023-02-04,-190.0,160.0,-13.0,-12.0,0.071429,0.25,...,0.340534,9.377817,8.230202,0.084156,1.733926,1.469282,0.253919,10.312010,16.058402,-0.040375


### Feature: UFC Bio Data

In [60]:
fighter_bios = pd.read_csv('data/final/aggregates/All_Fighter_Bios.csv')
fighter_bios.head(1)

,Unnamed: 0.1,Unnamed: 0,Status,Place of Birth,Fighting style,Age,Height,Weight,Octagon Debut,Reach,Leg reach,fighter,Trains at
0,0,1,Not Fighting,"Parrish, United States",MMA,32.0,72.0,155.0,"Jul. 30, 2019",78.0,42.0,Aalon Cruz,NaN


In [61]:
# if missing place of birth, replace with 'Unknown'
fighter_bios['Place of Birth'].fillna('Unknown', inplace=True)

# if missing fighting style, replace with 'Unknown'
fighter_bios['Fighting style'].fillna('Unknown', inplace=True)

# if missing Trains at, replace with 'Unknown'
fighter_bios['Trains at'].fillna('Unknown', inplace=True)

# if missing status, replace with 'Unknown'
fighter_bios['Status'].fillna('Unknown', inplace=True)

# if missing age, replace with median
fighter_bios['Age'].fillna(fighter_bios['Age'].median(), inplace=True)

# if missing height, replace with median -- thankfully not many missing, as 
# this is a somewhat important feature
fighter_bios['Height'].fillna(fighter_bios['Height'].median(), inplace=True)

# if missing weight, replace with median -- thankfully not many missing
fighter_bios['Weight'].fillna(fighter_bios['Weight'].median(), inplace=True)

In [62]:
def get_bio_data(fighter, stat):
    try:
        data = fighter_bios[fighter_bios['fighter'] == fighter]
        d = data[stat].values[0]
        d = pd.to_numeric(d)
        return d
    except:
        return nan

In [63]:
next_fight_df['A_Height'] = next_fight_df.apply(lambda row: get_bio_data(row['Fighter_A'], 'Height'), axis=1)
next_fight_df['B_Height'] = next_fight_df.apply(lambda row: get_bio_data(row['Fighter_B'], 'Height'), axis=1)
next_fight_df['Dif_Height'] = next_fight_df['A_Height'] - next_fight_df['B_Height']

next_fight_df['A_Reach'] = next_fight_df.apply(lambda row: get_bio_data(row['Fighter_A'], 'Reach'), axis=1)
next_fight_df['B_Reach'] = next_fight_df.apply(lambda row: get_bio_data(row['Fighter_B'], 'Reach'), axis=1)
next_fight_df['Dif_Reach'] = next_fight_df['A_Reach'] - next_fight_df['B_Reach']
next_fight_df['Reach_Dif'] = next_fight_df['A_Reach'] - next_fight_df['B_Reach']

next_fight_df['A_Leg_Reach'] = next_fight_df.apply(lambda row: get_bio_data(row['Fighter_A'], 'Leg reach'), axis=1)
next_fight_df['B_Leg_Reach'] = next_fight_df.apply(lambda row: get_bio_data(row['Fighter_B'], 'Leg reach'), axis=1)
next_fight_df['Dif_Leg_Reach'] = next_fight_df['A_Leg_Reach'] - next_fight_df['B_Leg_Reach']
next_fight_df['Leg_Reach_Dif'] = next_fight_df['A_Leg_Reach'] - next_fight_df['B_Leg_Reach']

In [64]:
# New column indicating if we have the reach data for fighter A
next_fight_df['A_Reach_NA'] = next_fight_df['A_Reach'].isna()

# New column indicating if we have the reach data for fighter B
next_fight_df['B_Reach_NA'] = next_fight_df['B_Reach'].isna()

# New column indicating if we have the reach data for either fighter
next_fight_df['Reach_NA'] = next_fight_df['A_Reach_NA'] | next_fight_df['B_Reach_NA']

# New column indicating if we have leg reach data for fighter A
next_fight_df['A_Leg_Reach_NA'] = next_fight_df['A_Leg_Reach'].isna()

# New column indicating if we have leg reach data for fighter B
next_fight_df['B_Leg_Reach_NA'] = next_fight_df['B_Leg_Reach'].isna()

# New column indicating if we have leg reach data for either fighter
next_fight_df['Leg_Reach_NA'] = next_fight_df['A_Leg_Reach_NA'] | next_fight_df['B_Leg_Reach_NA']

### Feature: Add Weightclass

In [65]:
all_events = pd.read_csv('data/final/events/All_Events_Fights_and_FightUrls.csv')

In [66]:
def find_typical_weightclass(fighter):
    try:
        data = all_events[all_events['Fighter1'] == fighter]
        data2 = all_events[all_events['Fighter2'] == fighter]
        data = pd.concat([data, data2])
        d = data['Weight class'].value_counts().index[0]
        return d
    except:
        return nan

In [67]:
next_fight_df['A_Typical_Weightclass'] = next_fight_df.apply(lambda row: find_typical_weightclass(row['Fighter_A']), axis=1)
next_fight_df['B_Typical_Weightclass'] = next_fight_df.apply(lambda row: find_typical_weightclass(row['Fighter_B']), axis=1)

In [68]:
# use fight_id to find weightclass of fight
def find_weightclass(fight_id):
    try:
        fight_url = 'http://www.ufcstats.com/fight-details/' + str(fight_id)
        data = all_events[all_events['fight_link'] == fight_url]
        d = data['Weight class'].values[0]
        return d
    except:
        return nan

In [69]:
next_fight_df['fight_weightclass'] = fights.apply(lambda row: find_weightclass(row['fight_id']), axis=1)

In [70]:
next_fight_df['A_Fight_in_Typical_Weightclass'] = next_fight_df.apply(lambda row: row['A_Typical_Weightclass'] == row['fight_weightclass'], axis=1)
next_fight_df['B_Fight_in_Typical_Weightclass'] = next_fight_df.apply(lambda row: row['B_Typical_Weightclass'] == row['fight_weightclass'], axis=1)

### Feature: Method

In [71]:
# split Method into Method and Method Detail
all_events['Method_Primary'] = all_events['Method'].str.split('  ', expand=True)[0]
all_events['Method_Detail'] = all_events['Method'].str.split('  ', expand=True)[1]
all_events.head(3)

,Unnamed: 0.1,Unnamed: 0,W/L,Weight class,Method,Round,Time,Fighter1,Fighter2,F1_Kd,...,F2_Str,F1_Td,F2_Td,F1_Sub,F2_Sub,fight_num,event_id,fight_link,Method_Primary,Method_Detail
0,0,0,win,Heavyweight,KO/TKO Punches,3,4:23,Ciryl Gane,Tai Tuivasa,1,...,29,0,0,0,0,1,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/810c9da0...,KO/TKO,Punches
1,1,1,win,Middleweight,U-DEC,3,5:00,Robert Whittaker,Marvin Vettori,0,...,33,1,0,0,0,2,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/b8ca1acd...,U-DEC,None
2,2,2,win,Middleweight,U-DEC,3,5:00,Nassourdine Imavov,Joaquin Buckley,0,...,46,2,0,2,0,3,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/20f316f9...,U-DEC,None


In [72]:
def get_method(fight_id):
    try:
        fight_url = 'http://www.ufcstats.com/fight-details/' + str(fight_id)
        data = all_events[all_events['fight_link'] == fight_url]
        d = data['Method_Primary'].values[0]
        return d
    except:
        return nan

In [73]:
def get_details(fight_id):
    try:
        fight_url = 'http://www.ufcstats.com/fight-details/' + str(fight_id)
        data = all_events[all_events['fight_link'] == fight_url]
        d = data['Method_Detail'].values[0]
        return d
    except:
        return nan

#### Feature: Round & Time

In [74]:
def get_round_time(fight_id, round_or_time):
    try: 
        fight_url = 'http://www.ufcstats.com/fight-details/' + str(fight_id)
        data = all_events[all_events['fight_link'] == fight_url]
        if round_or_time == 'round':
            d = data['Round'].values[0]
        elif round_or_time == 'time':
            d = data['Time'].values[0]
        return d
    except:
        return nan

### Feature: Size Ratios

In [75]:
next_fight_df['A_Ape_Index'] = next_fight_df['A_Reach'] / next_fight_df['A_Height']
next_fight_df['B_Ape_Index'] = next_fight_df['B_Reach'] / next_fight_df['B_Height']

# Leg_Index = Leg_Reach / Height
next_fight_df['A_Leg_Index'] = next_fight_df['A_Leg_Reach'] / next_fight_df['A_Height']
next_fight_df['B_Leg_Index'] = next_fight_df['B_Leg_Reach'] / next_fight_df['B_Height']

# Leg_to_Wing_Index = Leg_Reach / Reach
next_fight_df['A_Leg_to_Wing_Index'] = next_fight_df['A_Leg_Reach'] / next_fight_df['A_Reach']
next_fight_df['B_Leg_to_Wing_Index'] = next_fight_df['B_Leg_Reach'] / next_fight_df['B_Reach']

### Feature: Favorite

In [76]:
next_fight_df['favorite?'] = np.where(next_fight_df['Fighter_A_Odds'] < next_fight_df['Fighter_B_Odds'], 1, 0)

### Feature: Win/Loss Details

In [77]:
# add a formatted date column
next_fight_df['datetime'] = pd.to_datetime(next_fight_df['date'])
next_fight_df['date_formatted'] = next_fight_df['datetime'].dt.date

In [78]:
def get_number_UFC_fights(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    return len(data)


In [79]:
next_fight_df['A_Total_UFC_Fights'] = fights.apply(lambda row: get_number_UFC_fights(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Total_UFC_Fights'] = fights.apply(lambda row: get_number_UFC_fights(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Total_UFC_Fights'] = next_fight_df['A_Total_UFC_Fights'] - next_fight_df['B_Total_UFC_Fights']

### Wins

In [80]:
# add number of wins
def get_number_UFC_wins(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    return data['win?'].sum()

In [81]:
next_fight_df['A_UFC_Wins'] = fights.apply(lambda row: get_number_UFC_wins(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_UFC_Wins'] = fights.apply(lambda row: get_number_UFC_wins(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_UFC_Wins'] = next_fight_df['A_UFC_Wins'] - next_fight_df['B_UFC_Wins']

### Losses

In [82]:
def get_number_UFC_losses(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    return len(data) - data['win?'].sum()

In [83]:
next_fight_df['A_UFC_Losses'] = fights.apply(lambda row: get_number_UFC_losses(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_UFC_Losses'] = fights.apply(lambda row: get_number_UFC_losses(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_UFC_Losses'] = next_fight_df['A_UFC_Losses'] - next_fight_df['B_UFC_Losses']

### W/L Percentages

In [84]:
next_fight_df['A_UFC_Win_Percentage'] = next_fight_df['A_UFC_Wins'] / next_fight_df['A_Total_UFC_Fights']
next_fight_df['B_UFC_Win_Percentage'] = next_fight_df['B_UFC_Wins'] / next_fight_df['B_Total_UFC_Fights']

# add diff
next_fight_df['Dif_UFC_Win_Percentage'] = next_fight_df['A_UFC_Win_Percentage'] - next_fight_df['B_UFC_Win_Percentage']

### Last 5 Win Percentage

In [85]:
def get_last5_win_percentage(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    data = data.sort_values(by='datetime', ascending=False)
    data = data.head(5)
    return data['win?'].sum() / len(data)

In [86]:
next_fight_df['A_Last5_Win_Percentage'] = fights.apply(lambda row: get_last5_win_percentage(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Last5_Win_Percentage'] = fights.apply(lambda row: get_last5_win_percentage(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Last5_Win_Percentage'] = next_fight_df['A_Last5_Win_Percentage'] - next_fight_df['B_Last5_Win_Percentage']

### Last 3 Win Percentage

In [87]:
def get_last3_win_percentage(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    data = data.sort_values(by='datetime', ascending=False)
    data = data.head(3)
    return data['win?'].sum() / len(data)

In [88]:
next_fight_df['A_Last3_Win_Percentage'] = fights.apply(lambda row: get_last3_win_percentage(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Last3_Win_Percentage'] = fights.apply(lambda row: get_last3_win_percentage(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Last3_Win_Percentage'] = next_fight_df['A_Last3_Win_Percentage'] - next_fight_df['B_Last3_Win_Percentage']

### Win_by/ Loss_by Features

In [89]:
def get_general_method(fight_method):
    if 'DEC' in fight_method:
        return 'DEC'
    elif 'SUB' in fight_method:
        return 'SUB'
    elif 'KO' in fight_method:
        return 'KO'
    else:
        return 'OTHER'

In [90]:
def get_win_by_ko_percent(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    wins = data[data['win?'] == 1]
    wins = wins[wins['InFightData__General_Method'] == 'KO']
    return data['win?'].sum() / len(data)

def get_loss_by_ko_percent(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    losses = data[data['win?'] == 0]
    losses = losses[losses['InFightData__General_Method'] == 'KO']
    return data['win?'].sum() / len(data)

In [91]:
next_fight_df['A_Win_By_KO_Percentage'] = fights.apply(lambda row: get_win_by_ko_percent(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Win_By_KO_Percentage'] = fights.apply(lambda row: get_win_by_ko_percent(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Win_By_KO_Percentage'] = next_fight_df['A_Win_By_KO_Percentage'] - next_fight_df['B_Win_By_KO_Percentage']

In [92]:
next_fight_df['A_Loss_By_KO_Percentage'] = fights.apply(lambda row: get_loss_by_ko_percent(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Loss_By_KO_Percentage'] = fights.apply(lambda row: get_loss_by_ko_percent(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Loss_By_KO_Percentage'] = next_fight_df['A_Loss_By_KO_Percentage'] - next_fight_df['B_Loss_By_KO_Percentage']

### WL by decision

In [93]:
def get_win_by_decision_percent(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    wins = data[data['win?'] == 1]
    wins = wins[wins['InFightData__General_Method'] == 'DEC']
    return data['win?'].sum() / len(data)

def get_loss_by_decision_percent(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    losses = data[data['win?'] == 0]
    losses = losses[losses['InFightData__General_Method'] == 'DEC']
    return data['win?'].sum() / len(data)

In [94]:
next_fight_df['A_Win_By_Decision_Percentage'] = fights.apply(lambda row: get_win_by_decision_percent(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Win_By_Decision_Percentage'] = fights.apply(lambda row: get_win_by_decision_percent(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Win_By_Decision_Percentage'] = next_fight_df['A_Win_By_Decision_Percentage'] - next_fight_df['B_Win_By_Decision_Percentage']

In [95]:
next_fight_df['A_Loss_By_Decision_Percentage'] = fights.apply(lambda row: get_loss_by_decision_percent(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_Loss_By_Decision_Percentage'] = fights.apply(lambda row: get_loss_by_decision_percent(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_Loss_By_Decision_Percentage'] = next_fight_df['A_Loss_By_Decision_Percentage'] - next_fight_df['B_Loss_By_Decision_Percentage']

### Career Fight Time

In [96]:
def get_career_fight_time_seconds(fighter, date):
    data = fights[fights['Fighter_A'] == fighter]
    data = data[data['datetime'] < date]
    return data['InFightData__Total_Fight_Time_Seconds'].sum()

In [97]:
next_fight_df['A_UFC_Fight_Time_Seconds'] = fights.apply(lambda row: get_career_fight_time_seconds(row['Fighter_A'], row['datetime']), axis=1)
next_fight_df['B_UFC_Fight_Time_Seconds'] = fights.apply(lambda row: get_career_fight_time_seconds(row['Fighter_B'], row['datetime']), axis=1)

# add diff
next_fight_df['Dif_UFC_Fight_Time_Seconds'] = next_fight_df['A_UFC_Fight_Time_Seconds'] - next_fight_df['B_UFC_Fight_Time_Seconds']

In [98]:
next_fight_df

,Fighter_A,Fighter_B,fight_weightclass,date,Fighter_A_Odds,Fighter_B_Odds,Fighter_A_Odds_Change,Fighter_B_Odds_Change,A_Rolling_Kd_mean,B_Rolling_Kd_mean,...,Dif_Loss_By_KO_Percentage,A_Win_By_Decision_Percentage,B_Win_By_Decision_Percentage,Dif_Win_By_Decision_Percentage,A_Loss_By_Decision_Percentage,B_Loss_By_Decision_Percentage,Dif_Loss_By_Decision_Percentage,A_UFC_Fight_Time_Seconds,B_UFC_Fight_Time_Seconds,Dif_UFC_Fight_Time_Seconds
0,Derrick Lewis,Serghei Spivac,Women's Bantamweight,2023-02-04,190.0,-225.0,26.0,-21.0,0.363636,0.00,...,0.095238,0.666667,0.571429,0.095238,0.666667,0.571429,0.095238,7788,5595,2193
1,Da-Un Jung,Devin Clark,Heavyweight,2023-02-04,NaN,205.0,NaN,23.0,NaN,0.20,...,NaN,0.666667,NaN,NaN,0.666667,NaN,NaN,628,0,628
2,Marcin Tybura,Blagoy Ivanov,Heavyweight,2023-02-04,-190.0,160.0,-13.0,-12.0,0.071429,0.25,...,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1800,900,900
3,Dooho Choi,Kyle Nelson,Heavyweight,2023-02-04,NaN,180.0,NaN,-21.0,1.000000,0.00,...,-0.625000,0.000000,0.625000,-0.625000,0.000000,0.625000,-0.625000,1012,3250,-2238
4,Yusaku Kinoshita,Adam Fugitt,Welterweight,2023-02-04,-265.0,225.0,15.0,24.0,NaN,0.00,...,NaN,0.666667,NaN,NaN,0.666667,NaN,NaN,3704,0,3704
5,Jeka Saragih,Anshul Jubli,Welterweight,2023-02-04,-105.0,-115.0,73.0,-43.0,NaN,NaN,...,-0.400000,0.600000,1.000000,-0.400000,0.600000,1.000000,-0.400000,4812,282,4530
6,JeongYeong Lee,Yi Zha,Lightweight,2023-02-04,NaN,210.0,NaN,-10.0,NaN,NaN,...,-0.285714,0.714286,1.000000,-0.285714,0.714286,1.000000,-0.285714,3259,900,2359
7,Toshiomi Kazama,Rinya Nakamura,Featherweight,2023-02-04,330.0,-410.0,32.0,6.0,NaN,NaN,...,0.166667,0.500000,0.333333,0.166667,0.500000,0.333333,0.166667,1800,3897,-2097
8,SeungGuk Choi,HyunSung Park,Bantamweight,2023-02-04,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.750000,0.750000,0.000000,0.750000,0.750000,0.000000,3061,2616,445
9,Ji Yeon Kim,Mandy Bohm,Welterweight,2023-02-04,-285.0,240.0,-19.0,24.0,NaN,0.00,...,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,0,137,-137


### Stat/Round Features

In [99]:
topdown_cols = [n for n in next_fight_df.columns.to_list() if 'topdown' in n]
# A TOPDOWN
a_topdown_cols = [n for n in topdown_cols if n.startswith('A_')]
# get rid of any percent columns
a_topdown_cols = [n for n in a_topdown_cols if 'percent' not in n]

# B TOPDOWN
b_topdown_cols = [n for n in topdown_cols if n.startswith('B_')]
# get rid of any percent columns
b_topdown_cols = [n for n in b_topdown_cols if 'percent' not in n]
b_topdown_cols


['B_topdown_Avg_Kd',
 'B_topdown_Avg_Sig_strike_land',
 'B_topdown_Avg_Sig_strike_att',
 'B_topdown_Avg_Total_Strikes_land',
 'B_topdown_Avg_Total_Strikes_att',
 'B_topdown_Avg_Takedowns_land',
 'B_topdown_Avg_Takedowns_att',
 'B_topdown_Avg_Sub_Attempts_land',
 'B_topdown_Avg_Sub_Attempts_att',
 'B_topdown_Avg_Rev',
 'B_topdown_Avg_Ctrl_time_min',
 'B_topdown_Avg_Ctrl_time_sec',
 'B_topdown_Avg_Ctrl_time_tot',
 'B_topdown_Avg_Head_Strikes_land',
 'B_topdown_Avg_Head_Strikes_att',
 'B_topdown_Avg_Body_Strikes_land',
 'B_topdown_Avg_Body_Strikes_att',
 'B_topdown_Avg_Leg_Strikes_land',
 'B_topdown_Avg_Leg_Strikes_att',
 'B_topdown_Avg_Distance_Strikes_land',
 'B_topdown_Avg_Distance_Strikes_att',
 'B_topdown_Avg_Clinch_Strikes_land',
 'B_topdown_Avg_Clinch_Strikes_att',
 'B_topdown_Avg_Ground_Strikes_land',
 'B_topdown_Avg_Ground_Strikes_att']

In [100]:
len(b_topdown_cols)

25

In [101]:
len(a_topdown_cols)

25

In [102]:
next_fight_df['A_UFC_Fight_Rounds'] = next_fight_df['A_UFC_Fight_Time_Seconds'] / 300
next_fight_df['B_UFC_Fight_Rounds'] = next_fight_df['B_UFC_Fight_Time_Seconds'] / 300

In [103]:
# add per-round averages using topdown columns
for col in a_topdown_cols:
    next_fight_df[f'{col}_per_round'] = next_fight_df[col] / next_fight_df['A_UFC_Fight_Rounds']

for col in b_topdown_cols:
    next_fight_df[f'{col}_per_round'] = next_fight_df[col] / next_fight_df['B_UFC_Fight_Rounds']

In [104]:
next_fight_df

,Fighter_A,Fighter_B,fight_weightclass,date,Fighter_A_Odds,Fighter_B_Odds,Fighter_A_Odds_Change,Fighter_B_Odds_Change,A_Rolling_Kd_mean,B_Rolling_Kd_mean,...,B_topdown_Avg_Body_Strikes_land_per_round,B_topdown_Avg_Body_Strikes_att_per_round,B_topdown_Avg_Leg_Strikes_land_per_round,B_topdown_Avg_Leg_Strikes_att_per_round,B_topdown_Avg_Distance_Strikes_land_per_round,B_topdown_Avg_Distance_Strikes_att_per_round,B_topdown_Avg_Clinch_Strikes_land_per_round,B_topdown_Avg_Clinch_Strikes_att_per_round,B_topdown_Avg_Ground_Strikes_land_per_round,B_topdown_Avg_Ground_Strikes_att_per_round
0,Derrick Lewis,Serghei Spivac,Women's Bantamweight,2023-02-04,190.0,-225.0,26.0,-21.0,0.363636,0.00,...,0.017873,0.017873,0.000000,0.000000,1.000894,2.555853,0.053619,0.053619,1.090259,1.411975
1,Da-Un Jung,Devin Clark,Heavyweight,2023-02-04,NaN,205.0,NaN,23.0,NaN,0.20,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2,Marcin Tybura,Blagoy Ivanov,Heavyweight,2023-02-04,-190.0,160.0,-13.0,-12.0,0.071429,0.25,...,4.250000,4.666667,0.500000,0.500000,16.333333,34.750000,2.250000,3.166667,0.666667,0.750000
3,Dooho Choi,Kyle Nelson,Heavyweight,2023-02-04,NaN,180.0,NaN,-21.0,1.000000,0.00,...,0.876923,1.246154,0.461538,0.484615,2.630769,6.369231,0.600000,0.900000,0.069231,0.092308
4,Yusaku Kinoshita,Adam Fugitt,Welterweight,2023-02-04,-265.0,225.0,15.0,24.0,NaN,0.00,...,inf,inf,inf,inf,inf,inf,inf,inf,NaN,inf
5,Jeka Saragih,Anshul Jubli,Welterweight,2023-02-04,-105.0,-115.0,73.0,-43.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,JeongYeong Lee,Yi Zha,Lightweight,2023-02-04,NaN,210.0,NaN,-10.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Toshiomi Kazama,Rinya Nakamura,Featherweight,2023-02-04,330.0,-410.0,32.0,6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SeungGuk Choi,HyunSung Park,Bantamweight,2023-02-04,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Ji Yeon Kim,Mandy Bohm,Welterweight,2023-02-04,-285.0,240.0,-19.0,24.0,NaN,0.00,...,28.467153,33.941606,19.708029,21.897810,72.262774,192.700730,24.087591,26.277372,0.000000,1.094891


In [105]:
# get opponent average columns
opp_avg_cols = [n for n in next_fight_df.columns.to_list() if 'Opp_Avg' in n]
# A OPP AVG
a_opp_avg_cols = [n for n in opp_avg_cols if n.startswith('A_')]
# get rid of any percent columns
a_opp_avg_cols = [n for n in a_opp_avg_cols if 'percent' not in n]

# B OPP AVG
b_opp_avg_cols = [n for n in opp_avg_cols if n.startswith('B_')]
# get rid of any percent columns
b_opp_avg_cols = [n for n in b_opp_avg_cols if 'percent' not in n]

b_opp_avg_cols

['B_Opp_Avg_Kd',
 'B_Opp_Avg_Sig_strike_land',
 'B_Opp_Avg_Sig_strike_att',
 'B_Opp_Avg_Total_Strikes_land',
 'B_Opp_Avg_Total_Strikes_att',
 'B_Opp_Avg_Takedowns_land',
 'B_Opp_Avg_Takedowns_att',
 'B_Opp_Avg_Sub_Attempts_land',
 'B_Opp_Avg_Sub_Attempts_att',
 'B_Opp_Avg_Rev',
 'B_Opp_Avg_Ctrl_time_min',
 'B_Opp_Avg_Ctrl_time_sec',
 'B_Opp_Avg_Ctrl_time_tot',
 'B_Opp_Avg_Head_Strikes_land',
 'B_Opp_Avg_Head_Strikes_att',
 'B_Opp_Avg_Body_Strikes_land',
 'B_Opp_Avg_Body_Strikes_att',
 'B_Opp_Avg_Leg_Strikes_land',
 'B_Opp_Avg_Leg_Strikes_att',
 'B_Opp_Avg_Distance_Strikes_land',
 'B_Opp_Avg_Distance_Strikes_att',
 'B_Opp_Avg_Clinch_Strikes_land',
 'B_Opp_Avg_Clinch_Strikes_att',
 'B_Opp_Avg_Ground_Strikes_land',
 'B_Opp_Avg_Ground_Strikes_att']

In [106]:
# add per-round averages using opponent averages
for col in a_opp_avg_cols:
    next_fight_df[f'{col}_per_round'] = next_fight_df[col] / next_fight_df['A_UFC_Fight_Rounds']

for col in b_opp_avg_cols:
    next_fight_df[f'{col}_per_round'] = next_fight_df[col] / next_fight_df['B_UFC_Fight_Rounds']


#### Per Round Difs

In [107]:
# add difference in per-round averages

# get per-round topdown columns
topdown_per_round_cols = [n for n in next_fight_df.columns.to_list() if 'topdown' and 'per_round' in n]
# no difs
topdown_per_round_cols = [n for n in topdown_per_round_cols if 'Dif' not in n]

topdown_per_round_cols = pd.DataFrame(topdown_per_round_cols)
topdown_per_round_cols.columns = ['col']
topdown_per_round_cols['non-specific'] = topdown_per_round_cols['col'].str[2:]
# get unique values
topdown_per_round_cols = topdown_per_round_cols['non-specific'].unique().tolist()

In [108]:
for col in topdown_per_round_cols:
    next_fight_df[f'Dif_{col}'] = next_fight_df[f'A_{col}'] - next_fight_df[f'B_{col}']

In [109]:
next_fight_df

,Fighter_A,Fighter_B,fight_weightclass,date,Fighter_A_Odds,Fighter_B_Odds,Fighter_A_Odds_Change,Fighter_B_Odds_Change,A_Rolling_Kd_mean,B_Rolling_Kd_mean,...,Dif_Opp_Avg_Body_Strikes_land_per_round,Dif_Opp_Avg_Body_Strikes_att_per_round,Dif_Opp_Avg_Leg_Strikes_land_per_round,Dif_Opp_Avg_Leg_Strikes_att_per_round,Dif_Opp_Avg_Distance_Strikes_land_per_round,Dif_Opp_Avg_Distance_Strikes_att_per_round,Dif_Opp_Avg_Clinch_Strikes_land_per_round,Dif_Opp_Avg_Clinch_Strikes_att_per_round,Dif_Opp_Avg_Ground_Strikes_land_per_round,Dif_Opp_Avg_Ground_Strikes_att_per_round
0,Derrick Lewis,Serghei Spivac,Women's Bantamweight,2023-02-04,190.0,-225.0,26.0,-21.0,0.363636,0.00,...,-0.173976,-0.127428,0.112357,0.136870,-0.351953,-1.541100,0.060852,0.127388,0.182098,0.234627
1,Da-Un Jung,Devin Clark,Heavyweight,2023-02-04,NaN,205.0,NaN,23.0,NaN,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Marcin Tybura,Blagoy Ivanov,Heavyweight,2023-02-04,-190.0,160.0,-13.0,-12.0,0.071429,0.25,...,-2.523810,-3.785714,-3.607143,-4.226190,-10.261905,-24.702381,-2.892857,-4.440476,0.190476,0.309524
3,Dooho Choi,Kyle Nelson,Heavyweight,2023-02-04,NaN,180.0,NaN,-21.0,1.000000,0.00,...,5.548951,9.399149,0.104713,1.563849,11.584251,26.106294,0.562694,0.447309,1.194618,1.002888
4,Yusaku Kinoshita,Adam Fugitt,Welterweight,2023-02-04,-265.0,225.0,15.0,24.0,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Jeka Saragih,Anshul Jubli,Welterweight,2023-02-04,-105.0,-115.0,73.0,-43.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,JeongYeong Lee,Yi Zha,Lightweight,2023-02-04,NaN,210.0,NaN,-10.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Toshiomi Kazama,Rinya Nakamura,Featherweight,2023-02-04,330.0,-410.0,32.0,6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SeungGuk Choi,HyunSung Park,Bantamweight,2023-02-04,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Ji Yeon Kim,Mandy Bohm,Welterweight,2023-02-04,-285.0,240.0,-19.0,24.0,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Fix NAN and INF

In [110]:
# replace NANS with 0 for any column starting with A_ or B_
for col in next_fight_df.columns:
    if col.startswith('A_') or col.startswith('B_'):
        next_fight_df[col] = next_fight_df[col].fillna(0)

In [111]:
# replace inf with 0 for any columns starting with A_ or B_
for col in next_fight_df.columns:
    if col.startswith('A_') or col.startswith('B_'):
        next_fight_df[col] = next_fight_df[col].replace(np.inf, 0)

In [112]:
# re-calculate all dif columns

# get dif columns
dif_cols = [n for n in next_fight_df.columns.to_list() if 'Dif' in n]

dif_cols

['Dif_Rolling_Kd_mean',
 'Dif_Rolling_Sig_strike_land_mean',
 'Dif_Rolling_Sig_strike_att_mean',
 'Dif_Rolling_Sig_strike_percent_mean',
 'Dif_Rolling_Total_Strikes_land_mean',
 'Dif_Rolling_Total_Strikes_att_mean',
 'Dif_Rolling_Total_Strikes_percent_mean',
 'Dif_Rolling_Takedowns_land_mean',
 'Dif_Rolling_Takedowns_att_mean',
 'Dif_Rolling_Takedown_percent_mean',
 'Dif_Rolling_Sub_Attempts_land_mean',
 'Dif_Rolling_Sub_Attempts_att_mean',
 'Dif_Rolling_Rev_mean',
 'Dif_Rolling_Ctrl_time_min_mean',
 'Dif_Rolling_Ctrl_time_sec_mean',
 'Dif_Rolling_Ctrl_time_tot_mean',
 'Dif_Rolling_Head_Strikes_land_mean',
 'Dif_Rolling_Head_Strikes_att_mean',
 'Dif_Rolling_Head_Strikes_percent_mean',
 'Dif_Rolling_Body_Strikes_land_mean',
 'Dif_Rolling_Body_Strikes_att_mean',
 'Dif_Rolling_Body_Strikes_percent_mean',
 'Dif_Rolling_Leg_Strikes_land_mean',
 'Dif_Rolling_Leg_Strikes_att_mean',
 'Dif_Rolling_Leg_Strikes_percent_mean',
 'Dif_Rolling_Distance_Strikes_land_mean',
 'Dif_Rolling_Distance_Strik

In [113]:
dif_cols.remove('Reach_Dif')
dif_cols.remove('Leg_Reach_Dif')

In [114]:
for col in dif_cols:
    next_fight_df[col] = next_fight_df[f'A_{col[4:]}'] - next_fight_df[f'B_{col[4:]}']

In [115]:
next_fight_df['Reach_Dif'] = next_fight_df['A_Reach'] - next_fight_df['B_Reach']
next_fight_df['Leg_Reach_Dif'] = next_fight_df['A_Leg_Reach'] - next_fight_df['B_Leg_Reach']

In [116]:
next_fight_df

,Fighter_A,Fighter_B,fight_weightclass,date,Fighter_A_Odds,Fighter_B_Odds,Fighter_A_Odds_Change,Fighter_B_Odds_Change,A_Rolling_Kd_mean,B_Rolling_Kd_mean,...,Dif_Opp_Avg_Body_Strikes_land_per_round,Dif_Opp_Avg_Body_Strikes_att_per_round,Dif_Opp_Avg_Leg_Strikes_land_per_round,Dif_Opp_Avg_Leg_Strikes_att_per_round,Dif_Opp_Avg_Distance_Strikes_land_per_round,Dif_Opp_Avg_Distance_Strikes_att_per_round,Dif_Opp_Avg_Clinch_Strikes_land_per_round,Dif_Opp_Avg_Clinch_Strikes_att_per_round,Dif_Opp_Avg_Ground_Strikes_land_per_round,Dif_Opp_Avg_Ground_Strikes_att_per_round
0,Derrick Lewis,Serghei Spivac,Women's Bantamweight,2023-02-04,190.0,-225.0,26.0,-21.0,0.363636,0.00,...,-0.173976,-0.127428,0.112357,0.136870,-0.351953,-1.541100,0.060852,0.127388,0.182098,0.234627
1,Da-Un Jung,Devin Clark,Heavyweight,2023-02-04,NaN,205.0,NaN,23.0,0.000000,0.20,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Marcin Tybura,Blagoy Ivanov,Heavyweight,2023-02-04,-190.0,160.0,-13.0,-12.0,0.071429,0.25,...,-2.523810,-3.785714,-3.607143,-4.226190,-10.261905,-24.702381,-2.892857,-4.440476,0.190476,0.309524
3,Dooho Choi,Kyle Nelson,Heavyweight,2023-02-04,NaN,180.0,NaN,-21.0,1.000000,0.00,...,5.548951,9.399149,0.104713,1.563849,11.584251,26.106294,0.562694,0.447309,1.194618,1.002888
4,Yusaku Kinoshita,Adam Fugitt,Welterweight,2023-02-04,-265.0,225.0,15.0,24.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Jeka Saragih,Anshul Jubli,Welterweight,2023-02-04,-105.0,-115.0,73.0,-43.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,JeongYeong Lee,Yi Zha,Lightweight,2023-02-04,NaN,210.0,NaN,-10.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Toshiomi Kazama,Rinya Nakamura,Featherweight,2023-02-04,330.0,-410.0,32.0,6.0,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,SeungGuk Choi,HyunSung Park,Bantamweight,2023-02-04,NaN,NaN,NaN,NaN,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Ji Yeon Kim,Mandy Bohm,Welterweight,2023-02-04,-285.0,240.0,-19.0,24.0,0.000000,0.00,...,-77.737226,-108.394161,-7.664234,-20.802920,-97.445255,-274.817518,-30.656934,-41.605839,-9.854015,-16.423358


In [117]:
# get NAN
missing = next_fight_df.isna().sum()
missing = missing[missing > 0]
missing

Fighter_A_Odds           6
Fighter_B_Odds           2
Fighter_A_Odds_Change    5
Fighter_B_Odds_Change    2
dtype: int64

In [118]:
# replace unknown fighter odds with opposite of known fighter odds
next_fight_df['Fighter_A_Odds'] = next_fight_df['Fighter_A_Odds'].fillna(next_fight_df['Fighter_B_Odds'] * -1)
next_fight_df['Fighter_B_Odds'] = next_fight_df['Fighter_B_Odds'].fillna(next_fight_df['Fighter_A_Odds'] * -1)


In [119]:
# replace unknown fighter change odds with opposite of known fighter odds
next_fight_df['Fighter_A_Odds_Change'] = next_fight_df['Fighter_A_Odds_Change'].fillna(next_fight_df['Fighter_B_Odds_Change'] * -1)
next_fight_df['Fighter_B_Odds_Change'] = next_fight_df['Fighter_B_Odds_Change'].fillna(next_fight_df['Fighter_A_Odds_Change'] * -1)

In [120]:
# get NAN
missing = next_fight_df.isna().sum()
missing = missing[missing > 0]
missing

Fighter_A_Odds           2
Fighter_B_Odds           2
Fighter_A_Odds_Change    1
Fighter_B_Odds_Change    1
dtype: int64

In [121]:
# check difference in columns between next_fight_df and fights
# get columns in next_fight_df that are not in fights
next_fight_cols = next_fight_df.columns.to_list()
fights_cols = fights.columns.to_list()
missing_cols = [n for n in next_fight_cols if n not in fights_cols]
missing_cols

[]

In [122]:
# get the columns in fights that are not in next_fight_df
missing_cols = [n for n in fights_cols if n not in next_fight_cols]
missing_cols

['A_Kd',
 'B_Kd',
 'A_Sig_strike_land',
 'A_Sig_strike_att',
 'B_Sig_strike_land',
 'B_Sig_strike_att',
 'A_Sig_strike_percent',
 'B_Sig_strike_percent',
 'A_Total_Strikes_land',
 'A_Total_Strikes_att',
 'B_Total_Strikes_land',
 'B_Total_Strikes_att',
 'A_Total_Strikes_percent',
 'B_Total_Strikes_percent',
 'A_Takedowns_land',
 'A_Takedowns_att',
 'B_Takedowns_land',
 'B_Takedowns_att',
 'A_Takedown_percent',
 'B_Takedown_percent',
 'A_Sub_Attempts_land',
 'A_Sub_Attempts_att',
 'B_Sub_Attempts_land',
 'B_Sub_Attempts_att',
 'A_Rev',
 'B_Rev',
 'A_Ctrl_time_min',
 'A_Ctrl_time_sec',
 'B_Ctrl_time_min',
 'B_Ctrl_time_sec',
 'A_Ctrl_time_tot',
 'B_Ctrl_time_tot',
 'details',
 'event_title',
 'event_url',
 'Winner',
 'fight_id',
 'Unnamed: 0',
 'A_Head_Strikes_land',
 'A_Head_Strikes_att',
 'B_Head_Strikes_land',
 'B_Head_Strikes_att',
 'A_Head_Strikes_percent',
 'B_Head_Strikes_percent',
 'A_Body_Strikes_land',
 'A_Body_Strikes_att',
 'B_Body_Strikes_land',
 'B_Body_Strikes_att',
 'A_Bod

In [123]:
the_date = next_fight_df['date'].values[0]
the_date = pd.to_datetime(the_date).date()
# to str
the_date = the_date.strftime('%Y-%m-%d')
the_date

'2023-02-04'

In [124]:
# save to csv
next_fight_df.to_csv('data/final/next_fights/' + the_date + '.csv', index=False)

In [125]:
next_fight_df = pd.read_csv('data/final/next_fights/' + the_date + '.csv')


